In [1]:
import matplotlib
import seaborn
import numpy as np
from sklearn import datasets
import tarfile
import re
import os

Getting category names, i.e. folder names of 20 newsgroups folder

In [4]:
path = "20_newsgroups"
for root, dirs, files in os.walk(path, topdown=False):
    newsgroups = np.array(dirs)
print newsgroups

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


Reading files from first folder. Just use a for loop to read in from more folders.

In [3]:
path = "20_newsgroups" + "/" + newsgroups[0]
x = []
for root, dirs, files in os.walk(path):
    for name in files:
        f = open(path + "/" + name)
        x.append(f.read())
print x[1]

Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51060 alt.atheism.moderated:727 news.answers:7300 alt.answers:155
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!agate!netsys!ibmpcug!mantis!mathew
From: mathew <mathew@mantis.co.uk>
Newsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers
Subject: Alt.Atheism FAQ: Introduction to Atheism
Summary: Please read this file before posting to alt.atheism
Keywords: FAQ, atheism
Message-ID: <19930405122245@mantis.co.uk>
Date: Mon, 5 Apr 1993 12:22:45 GMT
Expires: Thu, 6 May 1993 12:22:45 GMT
Followup-To: alt.atheism
Distribution: world
Organization: Mantis Consultants, Cambridge. UK.
Approved: news-answers-request@mit.edu
Supersedes: <19930308134439@mantis.co.uk>
Lines: 646

Archive-name: atheism/introduction
Alt-atheism-archive-name: introduction
Last-modified: 5 April 1993
Version: 1.2

-----BEGIN PGP SIGNED MESSAGE-----

                          An Introduction t

## JunJie's Codes

In [ ]:
with tarfile.open("C:/Users/admin/Desktop/training/20news-19997.tar.gz", 'r:gz') as tf:
    print tf.getnames()
    # get information (metadata) about all files in the tarball
    file_infos = [file_info for file_info in tf if file_info.isfile()]
    
    Extract_msg=[]    
    
    for i in xrange(1000):
        # print one of them; for example, the first one
        message = tf.extractfile(file_infos[i]).read()
        Extract_msg.append(message)

In [ ]:
Compiled=[]

for message in Extract_msg:

    Content=message.split('\n\n')
    Email_content=''
    Meta_dict={}
   
    for index in xrange(len(Content)):
    
        if index == 0:
    
            Meta = sorted( Content[0].split('\n'))
            Meta_dict= { re.split(":\W" ,j)[0] : " ".join(re.split(":\W" ,j)[1:]) for j in Meta }

        elif len("".join(re.findall('[\-\-==]+',Content[index])))>80:
    
            break
        else:
            Email_content= Email_content + '\n'+ Content[index]    
        
    Meta_dict.update({'Content': Email_content})
    Compiled.append(Meta_dict)